In [1]:
import pickle
import numpy
with open("aa_sim_array.pickle", 'rb') as f:
    aa_mat = pickle.load(f)
with open("pos_sim_array.pickle", 'rb') as f:
    str_mat = pickle.load(f)
with open("pos_pdb_ids.pickle", 'rb') as f:
    ids = pickle.load(f)
with open("pos_labels.pickle", 'rb') as f:
    labels = pickle.load(f)

In [2]:
str_mat #Levenshtein distances - the bigger the less similar

array([[0.        , 0.41605839, 0.29927007, ..., 1.5212766 , 1.49468085,
        1.50531915],
       [0.        , 0.        , 0.41586538, ..., 1.53191489, 1.52659574,
        1.5212766 ],
       [0.        , 0.        , 0.        , ..., 1.56382979, 1.56382979,
        1.56382979],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.18617021,
        0.16489362],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.07978723],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [3]:
aa_mat #Levenshtein distances - the bigger the less similar

array([[0.        , 0.00235849, 0.00707547, ..., 1.65829146, 1.55399061,
        1.65829146],
       [0.        , 0.        , 0.00471698, ..., 1.65829146, 1.55868545,
        1.65829146],
       [0.        , 0.        , 0.        , ..., 1.65829146, 1.55868545,
        1.65829146],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.11557789,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.11557789],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [4]:
#interested in big aa difference, but small str difference -> aa_mat / str_mat -> the bigger the better (more interesting)

str_mat = str_mat + 0.00001
res = aa_mat / str_mat

In [5]:
import numpy as np
res = np.array(res)
np.max(res)

2127.659574468085

In [6]:
import bottleneck as bn

def top_n_indexes(arr, n):
    idx = bn.argpartition(arr, arr.size-n, axis=None)[-n:]
    width = arr.shape[1]
    return [divmod(i, width) for i in idx]

In [7]:
idx = top_n_indexes(res, 3000)
idx.sort(key = lambda tup: tup[0])

In [8]:
temp = np.partition(-res.flatten(), 5)
-temp[:5]

array([2127.65957447, 2127.65957447,   11.73043078,  513.69863014,
         10.94649216])

In [9]:
#sanity check
for i in idx:
    print(res[i[0], i[1]])

1.7563794071822074
2.0594428608453312
2.021655393387898
2.250503631569214
1.7473264300370597
1.7588081126126547
2.0871820448748917
2.127315421952427
2.120583499746237
1.768089320041157
1.772766753538294
1.7960545759871605
2.113894049677991
2.134090222075943
2.2633130760190445
2.027663521774009
2.072018022330792
2.068081800232553
2.0489920863392532
2.075438558976017
2.0371233136819393
2.108775930389517
2.025451612850823
2.0434106418581712
2.075438558976017
2.030874557525755
2.081965000712117
2.0805265750136814
2.0357158676418603
2.1233077692452818
2.0501703957518256
2.0501703957518256
2.0501703957518256
2.042769077553934
2.0067531768041875
2.0501703957518256
2.0501703957518256
2.0501703957518256
1.9070300687841106
1.9564551030683175
1.935493135014508
1.9070300687841106
2.8816537669577342
1.7504636272274716
1.759739618993701
1.7553806002215842
1.9027339039471232
1.8137193619076182
1.905570717500228
1.9227377872119524
1.9675924165423522
1.9282423547043128
1.939138790376835
2.0207828549066

In [53]:
#For twilight zone, we're only interested if aa distance is bigger than 0.75
count = 0
important_ids = []
important_labels = []
indices = []
used_idx= []
for i in idx:
    if (aa_mat[i[0], i[1]] > 0.7) and (str_mat[i[0], i[1]] < 0.25):
        important_ids.append(ids[i[0]])
        important_ids.append(ids[i[1]])
        important_labels.append(labels[i[0]])
        important_labels.append(labels[i[1]])
        indices.append(i[0])
        indices.append(i[1])
        used_idx.append((i[0], i[1]))
        count = count + 1
        print('"' + ids[i[0]] + '"' + ' paired with ' + '"' + ids[i[1]] + '"' + ' yields ' + str(res[i[0], i[1]]) + ' with aa distance ' + str(aa_mat[i[0], i[1]]) + ' and labels ' + labels[i[0]] + ' and ' + labels[i[1]])

"4PKM A 38:278" paired with "6RB9 A 34:260" yields 4.456516751916893 with aa distance 0.8152610441767069 and labels pos_18_ETX_MTX2.txt and pos_18_ETX_MTX2.txt
"4QX0 A 509:644" paired with "2L38 A 1:175" yields 3.9885376765703473 with aa distance 0.7377049180327869 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"4QX1 A 509:644" paired with "2L38 A 1:175" yields 3.0018631932819995 with aa distance 0.734251968503937 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"4QX2 A 509:644" paired with "2L38 A 1:175" yields 3.292018219862203 with aa distance 0.7591836734693878 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"4QX3 A 509:644" paired with "2L38 A 1:175" yields 3.2290276390877612 with aa distance 0.75 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin.txt
"1JI6 A 512:651" paired with "2L38 A 1:175" yields 3.4528113309634754 with aa distance 0.75 and labels pos_22_Thiol_cytolysin.txt and pos_22_Thiol_cytolysin

In [11]:
count

99

In [12]:
np.unique(np.array(important_ids))

array(['1JI6 A 512:651', '1O72 A 1:175', '1T5R A 29:275',
       '1XEZ A 190:452', '2I88 A 345:519', '2L38 A 1:175',
       '2LKF A 34:284', '2QQH A 237:460', '2RD7 A 237:460',
       '2ZS6 B 227:423', '3LIM A 3:178', '3LKF A 34:284',
       '3O44 A 190:452', '3OJY A 244:460', '3OJY B 219:442',
       '3ROH A 57:302', '3VWI A 3:178', '3ZWG A 3:178', '4E0S B 270:493',
       '4EN6 B 227:423', '4EN7 B 227:423', '4EN8 B 227:423',
       '4IYA A 29:275', '4IYC A 29:275', '4IZL A 29:275', '4MKO A 43:246',
       '4MKQ A 43:246', '4P1X B 29:275', '4PKM A 38:278',
       '4QX0 A 509:644', '4QX1 A 509:644', '4QX2 A 509:644',
       '4QX3 A 509:644', '5J67 A 802:916', '5J68 A 802:916',
       '5J69 A 802:915', '5J69 B 802:916', '5K59 C 62:305',
       '6FBM A 62:303', '6H03 B 291:514', '6H03 D 227:448',
       '6H03 F 237:460', '6H04 B 291:514', '6H04 D 227:448',
       '6H04 F 237:460', '6OQ6 A 1090:1431', '6RB9 A 34:260'],
      dtype='<U16')

In [13]:
len(np.unique(np.array(important_ids)))

47

In [14]:
np.unique(np.array(important_labels))

array(['pos_18_ETX_MTX2.txt', 'pos_22_Thiol_cytolysin.txt',
       'pos_37_Leukocidin.txt', 'pos_4_T4BSS_DotI_IcmL.txt',
       'pos_9_Bac_thur_toxin.txt'], dtype='<U26')

In [25]:
labels_from_indices = [labels[i] for i in np.unique(indices)]

In [26]:
from collections import Counter
print(Counter(labels_from_indices))

Counter({'pos_37_Leukocidin.txt': 29, 'pos_22_Thiol_cytolysin.txt': 10, 'pos_4_T4BSS_DotI_IcmL.txt': 4, 'pos_18_ETX_MTX2.txt': 2, 'pos_9_Bac_thur_toxin.txt': 2})


In [27]:
total = aa_mat.shape[0]

In [34]:
for i in range(total-1, 0, -1):
    for j in range(i-1, -1, -1):
        aa_mat[i,j] = aa_mat[j,i]

In [36]:
for i in range(total-1, 0, -1):
    for j in range(i-1, -1, -1):
        str_mat[i,j] = str_mat[j,i]

In [35]:
aa_mat[:5,:5]

array([[0.        , 0.00235849, 0.00707547, 0.11084906, 0.11320755],
       [0.00235849, 0.        , 0.00471698, 0.10849057, 0.11084906],
       [0.00707547, 0.00471698, 0.        , 0.11320755, 0.11556604],
       [0.11084906, 0.10849057, 0.11320755, 0.        , 0.00212766],
       [0.11320755, 0.11084906, 0.11556604, 0.00212766, 0.        ]])

In [39]:
#Sequence cluster
import umap
import umap.plot
import pandas as pd

mapper = umap.UMAP().fit(aa_mat)
hover_data = pd.DataFrame({'index':np.arange(total),
                           'label':labels})

hover_data['item'] = hover_data.label
hover_data['item']

C:\Users\sarva\Anaconda3\lib\site-packages\sklearn\manifold\_spectral_embedding.py:236: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


0          pos_10_Aerolysin.txt
1          pos_10_Aerolysin.txt
2          pos_10_Aerolysin.txt
3          pos_10_Aerolysin.txt
4          pos_10_Aerolysin.txt
                 ...           
367    pos_9_Bac_thur_toxin.txt
368    pos_9_Bac_thur_toxin.txt
369    pos_9_Bac_thur_toxin.txt
370    pos_9_Bac_thur_toxin.txt
371    pos_9_Bac_thur_toxin.txt
Name: item, Length: 372, dtype: object

In [43]:
from bokeh.plotting import show, save, output_notebook, output_file

p = umap.plot.interactive(mapper, labels=labels, hover_data=hover_data,point_size=20)
output_notebook()
show(p)

Loading BokehJS ...

In [46]:
#Structure cluster
mapper = umap.UMAP().fit(str_mat)
hover_data = pd.DataFrame({'index':np.arange(total),
                           'label':labels})

hover_data['item'] = hover_data.label
hover_data['item']

0          pos_10_Aerolysin.txt
1          pos_10_Aerolysin.txt
2          pos_10_Aerolysin.txt
3          pos_10_Aerolysin.txt
4          pos_10_Aerolysin.txt
                 ...           
367    pos_9_Bac_thur_toxin.txt
368    pos_9_Bac_thur_toxin.txt
369    pos_9_Bac_thur_toxin.txt
370    pos_9_Bac_thur_toxin.txt
371    pos_9_Bac_thur_toxin.txt
Name: item, Length: 372, dtype: object

In [47]:
from bokeh.plotting import show, save, output_notebook, output_file

p = umap.plot.interactive(mapper, labels=labels, hover_data=hover_data,point_size=20)
output_notebook()
show(p)

Loading BokehJS ...

In [91]:
#Number of islands problem - Leetcode 200 

def numIslands(grid):        
    count = 0
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i,j] == 1:
                dfs(grid, i, j)
                count += 1
    return count

def dfs(grid, i, j):
    if i<0 or j<0 or i>=len(grid) or j>=len(grid[0]) or grid[i,j] != 1:
        return
    grid[i][j] = -1
    dfs(grid, i+1, j)
    dfs(grid, i-1, j)
    dfs(grid, i, j+1)
    dfs(grid, i, j-1)

In [93]:
grid = np.zeros((total, total), dtype='int')

for i in used_idx:
    grid[i[0],i[1]] = 1

In [94]:
numIslands(grid)

40

In [88]:
grid2 = np.array([[1,1,0,0,0],[1,1,0,0,0],[0,0,1,0,0],[0,0,0,1,1]])

In [92]:
#Sanity check
numIslands(grid2)

3